In [1]:
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline 
import numpy as np
import pickle

In [2]:
# !pip install matplotlib

In [3]:
# from google.colab import drive
# drive.mount('/content/drive')


In [4]:
# import os
# os.chdir("/content/drive/")

In [5]:
# %cd MyDrive/Colab Notebooks/Room Classification/Train

In [6]:
from sklearn.model_selection import train_test_split
import tensorflow_hub as hub

In [7]:
labels = ['bathroom', 'bedroom', 'dining_room', 'exterior', 'interior', 'kitchen', 'living_room']

In [8]:
def training_batch(batch_folder, model_folder, batch_id):
    batch_path = batch_folder + '/batch_' + str(batch_id) + '.npz' 
    with np.load(batch_path) as batch:
        X = batch['X']
        y = batch['y']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    EfficientNet_B1_URL = "https://tfhub.dev/google/efficientnet/b1/feature-vector/1"
    pretrained_model_freezed = hub.KerasLayer(EfficientNet_B1_URL, input_shape=(240,240,3), trainable=False)
    model = keras.Sequential([
        pretrained_model_freezed,
        keras.layers.Dense(7, activation='softmax')

    ])
    model.compile(optimizer='adam',
             loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
             metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10)
    model.save(model_folder + '/model_' + str(batch_id) +'.h5')
    print('Model ' + str(batch_id) + ' trained and saved successfully!')

In [9]:
def training_batches(batch_folder, model_folder, num_batches):
    for i in range(1,num_batches+1):
        training_batch(batch_folder, model_folder, i)

In [10]:
training_batches('batches', 'bagging_models', 5)

Epoch 1/10
175/175 [==============================] - 99s 469ms/step - loss: 1.0772 - accuracy: 0.6444
Epoch 2/10
175/175 [==============================] - 83s 474ms/step - loss: 0.4858 - accuracy: 0.8414
Epoch 3/10
175/175 [==============================] - 83s 477ms/step - loss: 0.4289 - accuracy: 0.8549
Epoch 4/10
175/175 [==============================] - 83s 477ms/step - loss: 0.3863 - accuracy: 0.8685
Epoch 5/10
175/175 [==============================] - 84s 478ms/step - loss: 0.3609 - accuracy: 0.8812
Epoch 6/10
175/175 [==============================] - 84s 479ms/step - loss: 0.3362 - accuracy: 0.8898
Epoch 7/10
175/175 [==============================] - 84s 479ms/step - loss: 0.3122 - accuracy: 0.8885
Epoch 8/10
175/175 [==============================] - 84s 480ms/step - loss: 0.3060 - accuracy: 0.8976
Epoch 9/10
175/175 [==============================] - 84s 480ms/step - loss: 0.2868 - accuracy: 0.9111
Epoch 10/10
175/175 [==============================] - 84s 481ms/step - l

In [12]:
from tensorflow.keras.models import load_model

In [14]:
model_1 = load_model('bagging_models/model_1.h5', custom_objects={'KerasLayer': hub.KerasLayer})
model_1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              6575232   
_________________________________________________________________
dense (Dense)                (None, 7)                 8967      
Total params: 6,584,199
Trainable params: 8,967
Non-trainable params: 6,575,232
_________________________________________________________________


In [15]:
def model_evaluation(model_folder, batch_folder, batch_id):
    batch_path = batch_folder + '/batch_' + str(batch_id) + '.npz' 
    with np.load(batch_path) as batch:
        X = batch['X']
        y = batch['y']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = load_model('bagging_models/model_' + str(batch_id) + '.h5', custom_objects={'KerasLayer': hub.KerasLayer})
    model.evaluate(X_test, y_test)

In [16]:
for i in range(1, 6):
    model_evaluation('bagging_models', 'batches', i)

44/44 [==============================] - 22s 471ms/step - loss: 0.4537 - accuracy: 0.8407
